# Convergence of option prices for Euler and Milsten schemes

In [1]:
import numpy as np
import scipy.stats as st
import enum 

In [2]:
# This class defines puts and calls

class OptionType(enum.Enum):
    CALL = 1.0
    PUT = -1.0

In [3]:
# Black-Scholes call option price
def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    
    K = np.array(K).reshape([len(K),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    
    if CP == OptionType.CALL:
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    
    elif CP == OptionType.PUT:
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    
    return value

# Black-Scholes solution for cash or nothing option
def BS_Cash_Or_Nothing_Price(CP,S_0,K,sigma,tau,r):

    K = np.array(K).reshape([len(K),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    
    if CP == OptionType.CALL:
        value = K * np.exp(-r * tau) * st.norm.cdf(d2)
    
    if CP == OptionType.PUT:
        value = K * np.exp(-r * tau) *(1.0 - st.norm.cdf(d2))
    
    return value

In [4]:
def GeneratePathsGBMEuler(NoOfSteps, T, r, sigma, S_0, NoOfPaths = 1):    
    Z = np.random.normal(loc = 0.0, scale = 1.0, size = [NoOfPaths, NoOfSteps])
    W = np.zeros([NoOfPaths, NoOfSteps+1])
   
    # Approximation
    S = np.zeros([NoOfPaths, NoOfSteps+1])
    S[:,0] = S_0
    
    time = np.zeros([NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    
    for i in range(0,NoOfSteps):
        # Making sure that samples from a normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])

        W[:,i+1] = W[:,i] + np.power(dt, 0.5)*Z[:,i]
        
        S[:,i+1] = S[:,i] + r * S[:,i]* dt + sigma * S[:,i] * (W[:,i+1] - W[:,i])
        time[i+1] = time[i] + dt
        
    # Return S
    paths = {"time":time, "S":S}

    return paths

def GeneratePathsGBMMilstein(NoOfSteps, T, r, sigma, S_0, NoOfPaths = 1):    
    Z = np.random.normal(loc = 0.0, scale = 1.0, size = [NoOfPaths, NoOfSteps])
    W = np.zeros([NoOfPaths, NoOfSteps+1])
   
    # Approximation
    S = np.zeros([NoOfPaths, NoOfSteps+1])
    S[:,0] = S_0
        
    time = np.zeros([NoOfSteps+1])
        
    dt = T / float(NoOfSteps)
    
    for i in range(0,NoOfSteps):
        # Making sure that samples from a normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])
            
        W[:,i+1] = W[:,i] + np.power(dt, 0.5)*Z[:,i]
        
        S[:,i+1] = S[:,i] + r * S[:,i]* dt + sigma * S[:,i] * (W[:,i+1] - W[:,i]) \
                    + 0.5 * sigma * sigma * S[:,i] * (np.power((W[:,i+1] - W[:,i]),2) - dt)
        
    # Return S
    paths = {"time":time, "S":S}

    return paths

In [5]:
def EUOptionPriceFromMCPaths(CP,S,K,T,r):
    # S is a vector of Monte Carlo samples at T
    
    if CP == OptionType.CALL:
        return np.exp(-r*T) * np.mean(np.maximum(S-K,0.0))
    
    elif CP == OptionType.PUT:
        return np.exp(-r*T) * np.mean(np.maximum(K-S,0.0))

    
def CashofNothingPriceFromMCPaths(CP,S,K,T,r):
    # S is a vector of Monte Carlo samples at T

    if CP == OptionType.CALL:
        return np.exp(-r*T) * K * np.mean((S>K))
                                    
    elif CP == OptionType.PUT:
        return np.exp(-r*T) * K * np.mean((S<=K))

In [12]:
# Parameters

CP = OptionType.CALL
T = 1
r = 0.06
sigma = 0.3
S_0 = 5
K = [S_0]
m = 1000
# we consider several numbers of paths so to access convergence of MC for Euler and Milstein 
NoOfPathsV = [100,1000,5000,10000,50000,100000]

In [13]:
# Call price

print("EUROPEAN OPTION PRICING")
exactPrice = BS_Call_Option_Price(CP,S_0,K,sigma,T,r)[0]
print("Exact option price = {0}\n".format(exactPrice))

for NoOfPathsTemp in NoOfPathsV:
    np.random.seed(1)
    PathsEuler    = GeneratePathsGBMEuler(m,T,r,sigma,S_0, NoOfPathsTemp)
    S_Euler = PathsEuler["S"]
    
    np.random.seed(1)
    PathsMilstein = GeneratePathsGBMMilstein(m,T,r,sigma,S_0, NoOfPathsTemp)
    S_Milstein = PathsMilstein["S"]
    
    priceEuler = EUOptionPriceFromMCPaths(CP,S_Euler[:,-1],K,T,r)
    priceMilstein = EUOptionPriceFromMCPaths(CP,S_Milstein[:,-1],K,T,r)
    
    print("N = {0}".format(NoOfPathsTemp))
    print("Euler Scheme: option price = {0}, error = {1}".format(priceEuler,priceEuler-exactPrice))
    print("Milstein Scheme: option price = {0}, error = {1}\n".format(priceMilstein,priceMilstein-exactPrice))

EUROPEAN OPTION PRICING
Exact option price = [0.73585362]

N = 100
Euler Scheme: option price = 0.7285547612550968, error = [-0.00729886]
Milstein Scheme: option price = 0.728444051871542, error = [-0.00740957]

N = 1000
Euler Scheme: option price = 0.7200580888738554, error = [-0.01579553]
Milstein Scheme: option price = 0.7202098357606097, error = [-0.01564379]

N = 5000
Euler Scheme: option price = 0.7481360160804368, error = [0.0122824]
Milstein Scheme: option price = 0.7481404588408821, error = [0.01228684]

N = 10000
Euler Scheme: option price = 0.7387184512149807, error = [0.00286483]
Milstein Scheme: option price = 0.7386727436156081, error = [0.00281912]

N = 50000
Euler Scheme: option price = 0.7323620061398906, error = [-0.00349161]
Milstein Scheme: option price = 0.7323314739101704, error = [-0.00352215]

N = 100000
Euler Scheme: option price = 0.734484802469365, error = [-0.00136882]
Milstein Scheme: option price = 0.7344664631824348, error = [-0.00138716]



In [13]:
# Cash or nothing price

print("CASH OR NOTHING PRICING")
exactPrice = BS_Cash_Or_Nothing_Price(CP,S_0,K,sigma,T,r)
print("Exact option price = {0}".format(exactPrice))

for NoOfPathsTemp in NoOfPathsV:
    np.random.seed(2)
    PathsEuler    = GeneratePathsGBMEuler(m,T,r,sigma,S_0,NoOfPathsTemp)
    S_Euler = PathsEuler["S"]
    
    np.random.seed(2)
    PathsMilstein = GeneratePathsGBMMilstein(m,T,r,sigma,S_0,NoOfPathsTemp)
    S_Milstein = PathsMilstein["S"]
    
    priceEuler = CashofNothingPriceFromMCPaths(CP,S_Euler[:,-1],K[0],T,r)
    priceMilstein = CashofNothingPriceFromMCPaths(CP,S_Milstein[:,-1],K[0],T,r)
    
    print("N = {0}".format(NoOfPathsTemp))
    print("Euler Scheme: option price = {0}, error = {1}".format(priceEuler,priceEuler-exactPrice))
    print("Milstein Scheme: option price = {0}, error = {1}\n".format(priceMilstein,priceMilstein-exactPrice))

CASH OR NOTHING PRICING
Exact option price = [[2.44829963]]
N = 100
Euler Scheme: option price = 2.3544113339606216, error = [[-0.0938883]]
Milstein Scheme: option price = 2.3544113339606216, error = [[-0.0938883]]

N = 1000
Euler Scheme: option price = 2.4627142553228105, error = [[0.01441462]]
Milstein Scheme: option price = 2.4627142553228105, error = [[0.01441462]]

N = 5000
Euler Scheme: option price = 2.4853166041288324, error = [[0.03701697]]
Milstein Scheme: option price = 2.484374839595248, error = [[0.0360752]]

N = 10000
Euler Scheme: option price = 2.4782533701269505, error = [[0.02995374]]
Milstein Scheme: option price = 2.47589895879299, error = [[0.02759932]]

N = 50000
Euler Scheme: option price = 2.4529199041735343, error = [[0.00462027]]
Milstein Scheme: option price = 2.4525431983601003, error = [[0.00424356]]

N = 100000
Euler Scheme: option price = 2.450471316386215, error = [[0.00217168]]
Milstein Scheme: option price = 2.4496708165326684, error = [[0.00137118]]

